<a href="https://colab.research.google.com/github/AriKlein/intro_rl_foosball_lec2/blob/main/W3S2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Reinforcement learning with Foolsball
- Reinforcement learning is learning to make decisions from experience.
- Games are a good testbed for RL.
 

# About Foolsball
- 5x4 playground that provides a football/foosball-like environment.
- A controllable player:
  - always spawned in the top-left corner
  - displayed as '⚽'
  - can move North, South, East or West.
  - can be controlled algorithmically
- A number of **static** opponents, each represented by 👕, that occupy certain locations on the field.
- A goalpost 🥅 that is fixed in the bottom right corner

## Goals
### Primary goal
- We want the agent to learn to reach the goalpost 

### Secondary goals
- We may want the agent to learn to be efficient in some sense, for example, take the shortest path to the goalpost. 

## Rules 
- Initial rules:
    - The ball can be (tried to be) moved in four direction: \['n','e','w',s'\]
    - Move the ball to an unmarked position: -1 points
    - Move the ball to a position marked by a defender: -5 points
    - Try to move the ball ouside the field: -1 (ball stays in the previous position)
    - Move the ball into the goal post position: +5


# Create the enviroment

In [1]:
import numpy as np

agent = '⚽'
opponent = '👕'
goal = '🥅'

arena = [['⚽', ' ' , '👕', ' ' ],
         [' ' , ' ' , ' ' , '👕'],
         [' ' , '👕', ' ' , ' ' ],
         [' ' , ' ' , ' ' , '👕'],
         [' ' , '👕', ' ' , '🥅']]

In [2]:
class Foolsball(object):
    def __to_state__(self,row,col):
        """Convert from indices (row,col) to integer position."""
        return row*self.n_cols + col
    
    
    def __to_indices__(self, state):
        """Convert from inteeger position to indices(row,col)"""
        row = state // self.n_cols
        col = state % self.n_cols
        return row,col

    def __deserialize__(self,map:list,agent:str,opponent:str, goal:str):
        """Convrt a string representation of a map into a 2D numpy array
        Param map: list of lists of strings representing the player, opponents and goal.
        Param agent: string representing the agent on the map 
        Param opponent: string representing every instance of an opponent player
        Param goal: string representing the location of the goal on the map
        """
        ## Capture dimensions and map.
        self.n_rows = len(map)
        self.n_cols = len(map[0])
        self.n_states = self.n_rows * self.n_cols
        self.map = np.asarray(map)

        ## Store string representations for printing the map, etc.
        self.agent_repr = agent
        self.opponent_repr  = opponent
        self.goal_repr = goal

        ## Find initial state, the desired goal state and the state of the opponents. 
        self.init_state = None
        self.goal_state = None
        self.opponents_states = []

        for row in range(self.n_rows):
            for col in range(self.n_cols):
                if map[row][col] == agent:
                    # Store the initial state outside the map.
                    # This helps in quickly resetting the game to the initial state and
                    # also simplifies printing the map independent of the agent's state. 
                    self.init_state = self.__to_state__(row,col)
                    self.map[row,col] = ' ' 

                elif map[row][col] == opponent:
                    self.opponents_states.append(self.__to_state__(row,col))

                elif map[row][col] == goal:
                    self.goal_state = self.__to_state__(row,col)

        assert self.init_state is not None, f"Map {map} does not specify an agent {agent} location"
        assert self.goal_state is not None,  f"Map {map} does not specify a goal {goal} location"
        assert self.opponents_states,  f"Map {map} does not specify any opponents {opponent} location"

        return self.init_state
    
    
    def __init__(self,map,agent,opponent,goal):
        """Spawn the world, create variables to track state and actions."""
        # We just need to track the location of the agent (the ball)
        # Everything else is static and so a potential algorithm doesn't 
        # have to look at it. The variable `done` flags terminal states.
        self.state = self.__deserialize__(map,agent,opponent,goal)
        self.done = False
        self.actions = ['n','e','w','s']

        # Set up the rewards
        self.default_rewards = {'unmarked':-1, 'opponent':-5, 'outside':-1, 'goal':+5}
        self.set_rewards(self.default_rewards)
        
    def set_rewards(self,rewards):
        if not self.state == self.init_state:
            print('Warning: Setting reward while not in initial state! You may want to call reset() first.')
        for key in self.default_rewards:
            assert key in rewards, f'Key {key} missing from reward.'
        self.rewards = rewards
            
            
    def reset(self):
        """Reset the environment to its initial state."""
        # There's really just two things we need to reset: the state, which should
        # be reset to the initial state, and the `done` flag which should be 
        # cleared to signal that we are not in a terminal state anymore, even if we 
        # were earlier. 
        self.state = self.init_state
        self.done  = False
        return self.state
    
    def __get_next_state_on_action__(self,state,action):
        """Return next state based on current state and action."""
        assert not self.__is_terminal_state__(state), f"Action {action} undefined for terminal state {state}"
        
        row, col = self.__to_indices__(state)
        action_to_index_delta = {'n':[-1,0], 'e':[0,+1], 'w':[0,-1], 's':[+1,0]}

        row_delta, col_delta = action_to_index_delta[action]
        new_row , new_col = row+row_delta, col+col_delta

        ## Return current state if next state is invalid
        if not(0<=new_row<self.n_rows) or\
        not(0<=new_col<self.n_cols):
            return state  

        ## Construct state from new row and col and return it.    
        return self.__to_state__(new_row, new_col)    
    
  
    def __get_reward_for_transition__(self,state,next_state):
        """ Return the reward based on the transition from current state to next state. """
        ## Transition rejected due to illegal action (move)
        assert not self.__is_terminal_state__(state), f"Reward is undefined for terminal state {state}"
        
        if next_state == state:
            reward = self.rewards['outside']

        ## Goal!
        elif next_state == self.goal_state:
            reward = self.rewards['goal']

        ## Ran into opponent. 
        elif next_state in self.opponents_states:
            reward = self.rewards['opponent']

        ## Made a safe and valid move.   
        else:
            reward = self.rewards['unmarked']

        return reward    
    
    
    def __is_terminal_state__(self, state):
        return (state == self.goal_state) or (state in self.opponents_states) 
    
      
    def step(self,action):
        """Simulate state transition based on current state and action received."""
        assert not self.done, \
        f'You cannot call step() in a terminal state({self.state}). Check the "done" flag before calling step() to avoid this.'
        next_state = self.__get_next_state_on_action__(self.state, action)

        reward = self.__get_reward_for_transition__(self.state, next_state)

        done = self.__is_terminal_state__(next_state)

        self.state, self.done = next_state, done

        return next_state, reward, done
    
    
    
    def render(self):
        """Pretty-print the environment and agent."""
        ## Create a copy of the map and change data type to accomodate
        ## 3-character strings
        _map = np.array(self.map, dtype='<U3')

        ## Mark unoccupied positions with special symbol.
        ## And add extra spacing to align all columns.
        for row in range(_map.shape[0]):
            for col in range(_map.shape[1]):
                if _map[row,col] == ' ':
                    _map[row,col] = ' + '

                elif _map[row,col] == self.opponent_repr: 
                    _map[row,col] =  self.opponent_repr + ' '

                elif _map[row,col] == self.goal_repr:
                    _map[row,col] = ' ' + self.goal_repr + ' '

        ## If current state overlaps with the goal state or one of the opponents'
        ## states, susbstitute a distinct marker.
        if self.state == self.goal_state:
            r,c = self.__to_indices__(self.state)
            _map[r,c] = ' 🏁 '
        elif self.state in self.opponents_states:
            r,c = self.__to_indices__(self.state)
            _map[r,c] = ' ❗ '
        else:
            r,c = self.__to_indices__(self.state)
            _map[r,c] = ' ' + self.agent_repr

        for row in range(_map.shape[0]):
            for col in range(_map.shape[1]):
                print(f' {_map[row,col]} ',end="")
            print('\n') 

        print()


In [3]:
foolsball = Foolsball(arena, agent, opponent, goal)

In [4]:
foolsball.render()

  ⚽   +   👕    +  

  +    +    +   👕  

  +   👕    +    +  

  +    +    +   👕  

  +   👕    +    🥅  




# Override the default reward structure.
- Use a more sparse reward: {'unmarked':0, 'opponent':-5, 'outside':-1, 'goal':+5}

In [5]:
## Update reward structure to: {'unmarked':0, 'opponent':-5, 'outside':-1, 'goal':+5}
foolsball.reset()
foolsball.set_rewards({'unmarked':0, 'opponent':-5, 'outside':-1, 'goal':+5})

# Implement discounted returns¶
$$Discounted\ Return = R_{t_1} + \gamma*R_{t_2} + \gamma^2*R_{t_3} + ... + \gamma^{n-1}*R_{t_n}$$where $R_{t_k}$ is the reward after step k and $\gamma$ is called the discount factor.
- Set the discount factor $\gamma$ to 0.9

In [8]:
def get_discounted_return(path, gamma=0):
    foolsball.reset()
    foolsball.render()
    _return_ = 0
    discount_coeff = 1
    for act in path: 
        next_state, reward, done = foolsball.step(act)
        _return_ += discount_coeff*reward
        discount_coeff *= gamma    

        foolsball.render()
        if done:
            break
            
    print(f'Return (accumulated reward): {_return_}')

In [9]:
HYPER_PARAMS = {'gamma':0.9}

# Use dynamic programming to fill up the returns table
- The **highest discounted return** for a **(state, action)** can be defined in terms of returns of the next state.

$ Return(state_t,action_t) = Reward(state_t,state_{t+1}) + \gamma * \max \begin{bmatrix} Return(state_{t+1}, action_{t+1}=='n')\\ Return(state_{t+1}, action_{t+1}=='e')\\  Return(state_{t+1}, action_{t+1}=='w')\\  Return(state_{t+1}, action_{t+1}=='s') \end{bmatrix}$


In [10]:
import pandas as pd
def make_returns_table(states_list, actions_list, terminal_states):
    """Create an empty returns table where each entry is initialized arbitrarily."""
    table = pd.DataFrame.from_dict({s:{a:0 for a in actions_list} for s in states_list}, orient='index')
    return table

In [11]:
terminal_states = foolsball.opponents_states + [foolsball.goal_state]
RETURNS_TBL = make_returns_table(range(foolsball.n_states), foolsball.actions, terminal_states)
RETURNS_TBL

,n,e,w,s
0,0,0,0,0
1,0,0,0,0
2,0,0,0,0
3,0,0,0,0
4,0,0,0,0
5,0,0,0,0
6,0,0,0,0
7,0,0,0,0
8,0,0,0,0
9,0,0,0,0


In [12]:
def compute_returns(table,state,action, debug=False): 
    """ Recursively compute the discounted return for a (state,action) pair"""
    if not foolsball.__is_terminal_state__(state):

        next_state = foolsball.__get_next_state_on_action__(state, action)
        reward = foolsball.__get_reward_for_transition__(state, next_state)

        update = HYPER_PARAMS['gamma'] *\
        max(table.loc[next_state, foolsball.actions[0]],\
        table.loc[next_state, foolsball.actions[1]],\
        table.loc[next_state, foolsball.actions[2]],\
        table.loc[next_state, foolsball.actions[3]])

        table.loc[state, action]  = reward + update
    
    return table.loc[state,action]

In [13]:
for s in range(foolsball.n_states):
    for a in foolsball.actions:
        compute_returns(RETURNS_TBL,state=s, action=a, debug=True)
RETURNS_TBL

,n,e,w,s
0,-1.0,0.0,-1.0,0.0
1,-1.0,-5.0,0.0,0.0
2,0.0,0.0,0.0,0.0
3,-1.0,-1.0,-5.0,-5.0
4,0.0,0.0,-1.0,0.0
5,0.0,0.0,0.0,-5.0
6,-5.0,-5.0,0.0,0.0
7,0.0,0.0,0.0,0.0
8,0.0,-5.0,-1.0,0.0
9,0.0,0.0,0.0,0.0


# Let the returns stabilize (converge)¶

In [14]:
RETURNS_TBL = make_returns_table(range(foolsball.n_states), foolsball.actions, terminal_states)
for i in range(1,50):
    RETURNS_TBL_OLD = RETURNS_TBL.copy()
    for s in range(foolsball.n_states):
        for a in foolsball.actions:
            compute_returns(RETURNS_TBL,state=s, action=a, debug=True)
    
    if i%5 == 0:
        print(f'\n{i} iterations')
        print(RETURNS_TBL)
    
    deltas = RETURNS_TBL- RETURNS_TBL_OLD
    if abs(deltas.values.max()) < 1e-3:
        print(f'\nConvergence achieved at {i} iterations')
        print(RETURNS_TBL)
        break


5 iterations
          n       e        w        s
0  -1.00000  0.0000 -1.00000  0.00000
1  -1.00000 -5.0000  0.00000  0.00000
2   0.00000  0.0000  0.00000  0.00000
3  -4.09510 -4.0951 -5.00000 -5.00000
4   0.00000  0.0000 -1.00000  0.00000
5   0.00000  3.2805  0.00000 -5.00000
6  -5.00000 -5.0000  2.95245  3.64500
7   0.00000  0.0000  0.00000  0.00000
8   0.00000 -5.0000 -1.00000  3.28050
9   0.00000  0.0000  0.00000  0.00000
10  3.28050  3.2805 -5.00000  4.05000
11 -5.00000  2.2805  3.64500 -5.00000
12  2.95245  3.6450  2.28050  2.95245
13 -5.00000  4.0500  3.28050 -5.00000
14  3.64500 -5.0000  3.64500  4.50000
15  0.00000  0.0000  0.00000  0.00000
16  3.28050 -5.0000  1.95245  1.95245
17  0.00000  0.0000  0.00000  0.00000
18  4.05000  5.0000 -5.00000  3.50000
19  0.00000  0.0000  0.00000  0.00000

Convergence achieved at 9 iterations
           n         e         w         s
0   1.391485  2.657205  1.391485  2.657205
1   1.657205 -5.000000  2.391485  2.952450
2   0.000000  0.00000

# Intro to Policies

In [15]:
def greedy_policy_from_returns_tbl(table):
    policy = {s:None for s in table.index }
    for state in table.index:
        if state not in terminal_states:
            greedy_action = table.loc[state].idxmax()
            policy[state] = greedy_action
            
    return policy

In [16]:
policy0 = greedy_policy_from_returns_tbl(RETURNS_TBL)
policy0

{0: 'e',
 1: 's',
 2: None,
 3: 'w',
 4: 'e',
 5: 'e',
 6: 's',
 7: None,
 8: 's',
 9: None,
 10: 's',
 11: 'w',
 12: 'e',
 13: 'e',
 14: 's',
 15: None,
 16: 'n',
 17: None,
 18: 'e',
 19: None}

In [17]:
def pretty_print_policy(policy):
    direction_repr = {'n':' 🡑 ', 'e':' 🡒 ', 'w':' 🡐 ', 's':' 🡓 ', None:' ⬤ '}

    for row in range(foolsball.n_rows):
        for col in range(foolsball.n_cols):
            state = foolsball.__to_state__(row, col)
            print(direction_repr[policy[state]],end='')
        print()

In [18]:
pretty_print_policy(policy0)

 🡒  🡓  ⬤  🡐 
 🡒  🡒  🡓  ⬤ 
 🡓  ⬤  🡓  🡐 
 🡒  🡒  🡓  ⬤ 
 🡑  ⬤  🡒  ⬤ 


# Dealing with incomplete Knowledge of the environment

In [21]:
import numpy as np
def collect_random_episode():
    state = foolsball.reset()
    done = False
    episode = []
    
    while not done:
        action = np.random.choice(foolsball.actions)
        next_state, reward, done = foolsball.step(action)
        episode.append([state, action, reward])
        state = next_state
        
    return episode

In [22]:
ep = collect_random_episode()
foolsball.render()
print(ep)

  +    +    ❗    +  

  +    +    +   👕  

  +   👕    +    +  

  +    +    +   👕  

  +   👕    +    🥅  


[[0, 'w', -1], [0, 'e', 0], [1, 'e', -5]]


# Implement discounted returns for episodes
- If an episode is: (s1,a1,r1),(s2,a2,r2),(s3,a3,r3), (s4),  s4 being a terminal state:
  - The (discounted) return for (s1,a1) is r1+γ∗r2+γ2∗r3
  - The (discounted) return for (s2, a2)is r2+γ∗r3
  - The (discounted) return for (s3,a3) is r3


In [23]:
def discounted_return_from_episode(ep, gamma=0):
    states, actions, rewards = list(zip(*ep))
    rewards = np.asarray(rewards)
    discount_coeffs = np.asarray([np.power(gamma,p) for p in range(len(rewards))])
    
    l = len(rewards)
    discounted_returns = [np.dot(rewards[i:],discount_coeffs[:l-i]) for i in range(l)]
    
    return (states, actions, discounted_returns)

In [24]:
discounted_return_from_episode(ep, gamma=HYPER_PARAMS['gamma'])

((0, 0, 1), ('w', 'e', 'e'), [-5.050000000000001, -4.5, -5.0])

# Exploration-Exploitation with Epsilon Decay

In [25]:
def collect_epsilon_greedy_episode_from_returns_tbl(table, max_ep_len=20, epsilon=0.1):
    state = foolsball.reset()
    done = False
    episode = []
    
    for _ in range(max_ep_len):
        if done:
            break
    
        actions = table.columns
        action_probs = np.asarray([epsilon/len(actions)]*len(actions),dtype=np.float)

        greedy_action_index = np.argmax(table.loc[state].values)
        action_probs[greedy_action_index] += 1-epsilon

        epsilon_greedy_action = np.random.choice(table.columns,p=action_probs)

        next_state, reward, done = foolsball.step(epsilon_greedy_action)
        episode.append([state, epsilon_greedy_action, reward])
        state = next_state

    return episode

In [26]:
ESTIMATED_RETURNS_TBL = pd.DataFrame.from_dict({s:{a:0 for a in foolsball.actions} for s in range(foolsball.n_states)}, orient='index')
VISITS_COUNTS_TBL = pd.DataFrame.from_dict({s:{a:0 for a in foolsball.actions} for s in range(foolsball.n_states)}, orient='index')

n_episodes = 5000
epsilon = 1
min_epsilon = 0.1
epsilon_decay = 0.999

for i in range(n_episodes):
    estimated_returns = ESTIMATED_RETURNS_TBL.div(VISITS_COUNTS_TBL+1)
  
    epsilon = max(epsilon, min_epsilon)
    episode_i = collect_epsilon_greedy_episode_from_returns_tbl(estimated_returns,epsilon=epsilon)
    epsilon *= epsilon_decay
    #print(episode_i)
    states, actions, discounted_returns = discounted_return_from_episode(episode_i, gamma=HYPER_PARAMS['gamma'])

    for s,a,ret in zip(states, actions, discounted_returns):
        ESTIMATED_RETURNS_TBL.loc[s,a] += ret
        VISITS_COUNTS_TBL.loc[s,a] += 1

In [27]:
estimated_returns = ESTIMATED_RETURNS_TBL.div(VISITS_COUNTS_TBL+1) ## Averaging returns. Avoid dividing by zeros.
print(estimated_returns)

policy4 = greedy_policy_from_returns_tbl(estimated_returns)
print(policy4)

pretty_print_policy(policy4)

           n         e         w         s
0  -2.029627 -1.288397 -2.008482 -0.378704
1  -2.992107 -4.978632 -1.964128 -0.915024
2   0.000000  0.000000  0.000000  0.000000
3   0.000000  0.000000  0.000000  0.000000
4  -0.364071 -1.086082 -1.836632 -1.051419
5  -1.783415 -2.055021 -0.640600 -4.982079
6  -4.880952 -4.878049 -1.315188 -2.118011
7   0.000000  0.000000  0.000000  0.000000
8  -0.723996 -4.970414 -2.711965 -1.015609
9   0.000000  0.000000  0.000000  0.000000
10 -2.720409 -2.130805 -4.166667 -0.765638
11 -3.750000 -3.816667 -1.169471 -3.750000
12 -2.110119 -0.175032 -2.910631 -2.656098
13 -4.500000  1.007316 -1.767413 -4.375000
14 -0.127181 -4.545455 -2.356286  1.904575
15  0.000000  0.000000  0.000000  0.000000
16 -1.362556 -4.615385 -3.198493 -3.844303
17  0.000000  0.000000  0.000000  0.000000
18 -0.121500  4.880952 -4.545455  3.111111
19  0.000000  0.000000  0.000000  0.000000
{0: 's', 1: 's', 2: None, 3: 'n', 4: 'n', 5: 'w', 6: 'w', 7: None, 8: 'n', 9: None, 10: 's', 11: 

# Constant Alpha

## The idea:
- Dividing the accumulated returns by visit count has a non linear effect on the updates. (Go back to previous step and see for yourself).
- Don't divide at all!
- But we need to ensure that updates are small
- Idea:
 - ESTIMATED_RETURNS_TBL.loc[s,a] and ret are both estimates of the same quantity.
 - Use the difference of the two estimates to update ESTIMATED_RETURNS_TBL.loc[s,a] much like we do in Deep Learning.

Todo:
- Complete the missing code in the next cell.
- Run the next few cells to get a policy and evaluate it.
- Does the policy help the agent attain its goal?



In [56]:
# SARSA

ESTIMATED_RETURNS_TBL = pd.DataFrame.from_dict({s:{a:0 for a in foolsball.actions} for s in range(foolsball.n_states)}, orient='index')

n_episodes = 5000
epsilon = 1
min_epsilon = 0.1
epsilon_decay = 0.999

alpha = 0.001

table = ESTIMATED_RETURNS_TBL

i = 1
done  = False
s_t = foolsball.reset()
a_t = np.random.choice(table.columns)

while i < n_episodes:
  
    if done:
        #foolsball.reset()
        s_t = foolsball.reset()
        done = False
        epsilon *= epsilon_decay
        i += 1
        epsilon = max(epsilon,min_epsilon)

        actions = table.columns
        action_probs = np.asarray([epsilon/len(actions)]*len(actions),dtype=np.float)

        greedy_action_index = np.argmax(table.loc[s_t].values)
        action_probs[greedy_action_index] += 1-epsilon

        a_t = np.random.choice(table.columns,p=action_probs)

    epsilon = max(epsilon,min_epsilon)

    s_tp1, r_tp1, done = foolsball.step(a_t)

    actions = table.columns
    action_probs = np.asarray([epsilon/len(actions)]*len(actions),dtype=np.float)

    greedy_action_index = np.argmax(table.loc[s_tp1].values)
    action_probs[greedy_action_index] += 1-epsilon

    a_tp1 = np.random.choice(table.columns,p=action_probs)


    gamma=HYPER_PARAMS['gamma']

    # SARSA
    ESTIMATED_RETURNS_TBL.loc[s_t, a_t] += alpha*(r_tp1 + gamma*ESTIMATED_RETURNS_TBL.loc[s_tp1,a_tp1]- ESTIMATED_RETURNS_TBL.loc[s_t,a_t])

    s_t = s_tp1
    a_t = a_tp1


In [58]:
# Q-learning

ESTIMATED_RETURNS_TBL = pd.DataFrame.from_dict({s:{a:0 for a in foolsball.actions} for s in range(foolsball.n_states)}, orient='index')

n_episodes = 5000
epsilon = 1
min_epsilon = 0.1
epsilon_decay = 0.999

alpha = 0.001

table = ESTIMATED_RETURNS_TBL

i = 1
done  = False
s_t = foolsball.reset()
a_t = np.random.choice(table.columns)

while i < n_episodes:
    
    if done: # start new episode
        #foolsball.reset()
        s_t = foolsball.reset()
        done = False
        if i>500: # do exploration only until 500th episode
          epsilon *= epsilon_decay
        i += 1
        epsilon = max(epsilon,min_epsilon)
        #episode_i = collect_epsilon_greedy_episode_from_returns_tbl(estimated_returns,epsilon=epsilon)

        actions = table.columns
        action_probs = np.asarray([epsilon/len(actions)]*len(actions),dtype=np.float)

        greedy_action_index = np.argmax(table.loc[s_t].values)
        action_probs[greedy_action_index] += 1-epsilon

        a_t = np.random.choice(table.columns,p=action_probs)

    epsilon = max(epsilon,min_epsilon)

    actions = table.columns
    action_probs = np.asarray([epsilon/len(actions)]*len(actions),dtype=np.float)

    greedy_action_index = np.argmax(table.loc[s_t].values)
    action_probs[greedy_action_index] += 1-epsilon

    a_t = np.random.choice(table.columns,p=action_probs)

    gamma=HYPER_PARAMS['gamma']

    s_tp1, r_tp1, done = foolsball.step(a_t)

    # Q-learning
    max_return = table.loc[s_tp1].max()
    ESTIMATED_RETURNS_TBL.loc[s_t, a_t] += alpha*(r_tp1 + gamma*max_return- ESTIMATED_RETURNS_TBL.loc[s_t,a_t])

    s_t = s_tp1


In [59]:
estimated_returns = ESTIMATED_RETURNS_TBL
print(estimated_returns)

policy5 = greedy_policy_from_returns_tbl(estimated_returns)
print(policy5)

pretty_print_policy(policy5)

               n             e             w             s
0  -6.106818e-01  1.144246e-01 -6.276689e-01  5.843421e-04
1  -4.266872e-01 -2.170349e+00  1.649476e-03  2.869030e-01
2   0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00
3   0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00
4   3.969400e-05  2.878204e-02 -3.364568e-01  7.852333e-10
5   5.329032e-03  6.499995e-01  4.182445e-04 -1.594997e+00
6  -1.145244e+00 -1.083040e+00  1.070606e-02  1.289193e+00
7   0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00
8   1.753846e-05 -5.922037e-01 -1.530241e-01  3.456395e-14
9   0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00
10  2.352119e-02  1.343163e-03 -8.614588e-01  2.290378e+00
11 -9.905568e-02 -1.580721e-02  6.607392e-02 -1.381263e-01
12  5.208108e-09  4.610568e-21 -6.950263e-02  5.496647e-20
13 -1.235114e-01  1.165058e-01  2.617301e-12 -1.478452e-01
14  6.744956e-02 -7.693555e-01  3.054988e-03  3.522150e+00
15  0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+

# How can we get faster convergence?

- Try the SARSA and Q-learning appraches described [here](https://lilianweng.github.io/lil-log/2018/02/19/a-long-peek-into-reinforcement-learning.html#sarsa-on-policy-td-control) 